##Environment Setup

In [6]:
# # Mounting google drive

from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
# Standard library
import os
import matplotlib.pyplot as plt
from PIL import Image, UnidentifiedImageError

# Third-party
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Subset


##Kaggle API setup

In [ ]:
!pip install kaggle

# Upload kaggle.json and configure Kaggle API
from google.colab import files
files.upload()  # Upload kaggle.json manually

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download and unzip dataset from Kaggle
!mkdir -p /content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd
!kaggle datasets download -d ismailnasri20/driver-drowsiness-dataset-ddd -p /content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd --unzip


##Data Loading & Inspection

In [ ]:
dataset_path = '/content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/'

def show_first_5_images(folder_name):
    folder_path = os.path.join(dataset_path, folder_name)
    image_files = os.listdir(folder_path)[:5]  # first 5 image filenames

    plt.figure(figsize=(15, 5))
    for i, image_file in enumerate(image_files):
        img_path = os.path.join(folder_path, image_file)
        img = Image.open(img_path)

        plt.subplot(1, 5, i+1)
        plt.imshow(img)
        plt.title(f"{folder_name} {i+1}")
        plt.axis('off')
    plt.show()

# Show first 5 images from 'drowsy' folder
show_first_5_images('Drowsy')

# Show first 5 images from 'notdrowsy' folder
show_first_5_images('Non Drowsy')

##Data Quality Check

In [ ]:
def check_images(root_dir):
    for root, _, files in os.walk(root_dir):
        for file in files:
            path = os.path.join(root, file)
            try:
                img = Image.open(path)
                img.verify()
            except Exception:
                print(f"Corrupted: {path}")
                # Optional: os.remove(path)

check_images("/content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/")#B0084.png was removed


##Transform, Dataset Split

In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset_path = '/content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset (DDD)/'
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])




train_dataset = Subset(train_dataset, range(1000))
val_dataset = Subset(val_dataset, range(200))



# Define DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Train Function

In [20]:
def train_fn(index, total_epochs=10):

    global train_dataset, val_dataset
    train_dataset = Subset(train_dataset, range(1000))
    val_dataset = Subset(val_dataset, range(200))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



    model = resnet50(weights=ResNet50_Weights.DEFAULT)

    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, 2)
    for param in model.fc.parameters():
        param.requires_grad = True

    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    save_dir = "/content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd/checkpoints"
    os.makedirs(save_dir, exist_ok=True)
    checkpoint_path = os.path.join(save_dir, f"resnet50_gpu_{index}_checkpoint.pth")

    start_epoch = 0
    best_acc = 0.0

    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_acc = checkpoint.get('best_acc', 0.0)
        print(f"[gpu:{index}] Resuming from epoch {start_epoch} with best acc {best_acc:.4f}")

    for epoch in range(start_epoch, total_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            if i % 20 == 0:
                print(f"[gpu:{index}] Epoch {epoch+1} | Batch {i}/{len(train_loader)} | Loss: {loss.item():.4f}")

        epoch_loss = total_loss / total
        epoch_acc = correct / total
        print(f"[gpu:{index}] Epoch {epoch+1} - Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.4f}")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        val_loss = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        val_epoch_loss = val_loss / val_total
        val_epoch_acc = val_correct / val_total
        print(f"[gpu:{index}] Epoch {epoch+1} - Val Loss: {val_epoch_loss:.4f} - Val Acc: {val_epoch_acc:.4f}")

        # Save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': max(best_acc, val_epoch_acc)
        }, checkpoint_path)

        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            best_model_path = os.path.join(save_dir, f"best_model_gpu_{index}.pth")
            torch.save(model.state_dict(), best_model_path)
            print(f"[gpu:{index}] Best model saved with acc {best_acc:.4f}")


## Start Training

In [21]:
train_fn(0)

[gpu:0] Resuming from epoch 2 with best acc 0.7600
[gpu:0] Epoch 3 | Batch 0/32 | Loss: 0.6500
[gpu:0] Epoch 3 | Batch 20/32 | Loss: 0.6664
[gpu:0] Epoch 3 - Train Loss: 0.6271 - Train Acc: 0.7680
[gpu:0] Epoch 3 - Val Loss: 0.6206 - Val Acc: 0.8200
[gpu:0] Best model saved with acc 0.8200
[gpu:0] Epoch 4 | Batch 0/32 | Loss: 0.6297
[gpu:0] Epoch 4 | Batch 20/32 | Loss: 0.5927
[gpu:0] Epoch 4 - Train Loss: 0.6066 - Train Acc: 0.7820
[gpu:0] Epoch 4 - Val Loss: 0.5950 - Val Acc: 0.8350
[gpu:0] Best model saved with acc 0.8350
[gpu:0] Epoch 5 | Batch 0/32 | Loss: 0.6085
[gpu:0] Epoch 5 | Batch 20/32 | Loss: 0.6044
[gpu:0] Epoch 5 - Train Loss: 0.5841 - Train Acc: 0.8300
[gpu:0] Epoch 5 - Val Loss: 0.5759 - Val Acc: 0.8700
[gpu:0] Best model saved with acc 0.8700
[gpu:0] Epoch 6 | Batch 0/32 | Loss: 0.5946
[gpu:0] Epoch 6 | Batch 20/32 | Loss: 0.5758
[gpu:0] Epoch 6 - Train Loss: 0.5673 - Train Acc: 0.8390
[gpu:0] Epoch 6 - Val Loss: 0.5643 - Val Acc: 0.8550
[gpu:0] Epoch 7 | Batch 0/32 |

##Evaluation


In [22]:
def load_best_model(index):
    from torchvision.models import resnet50, ResNet50_Weights
    import torch.nn as nn
    import torch

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, 2)  # Match your output layer
    model = model.to(device)

    best_model_path = f"/content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd/checkpoints/best_model_gpu_{index}.pth"
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()

    return model.to(device)


In [23]:
from sklearn.metrics import classification_report, confusion_matrix
import torch

def evaluate(model, dataloader, device):
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_preds, digits=4))

    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))


In [24]:
model = load_best_model(index=0)


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
evaluate(model, test_loader, device)

##Deployment

In [ ]:
# Load your best model
model = load_best_model(index=0)
model.eval()

# Example input to trace model structure
example_input = torch.randn(1, 3, 224, 224).to(next(model.parameters()).device)

# Convert to TorchScript
traced_script_module = torch.jit.trace(model, example_input)

export_path = "/content/drive/MyDrive/kaggle_datasets/driver-drowsiness-dataset-ddd/deployed-models"

# TorchScript export
traced_script_module.save(f"{export_path}/resnet50_finetuned_script.pt")


